In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from copy import deepcopy
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import CubicSpline
import warnings
warnings.filterwarnings("ignore")
import arviz as az
%run -i '../../../miscellaneous/utils.py'

# Read data

In [2]:
df_bac = pd.read_excel('16S_relative_abundance.xlsx', index_col=0)
df_meta = pd.read_csv('metadata.csv', index_col=0)

# Generate stan program files

In [3]:
bacterial_taxa = generate_stan_files_for_fiber_respones(
    df_bac, # 16S data (relative or absolute), rows are samples, columns are taxa
    df_meta, # meta data, rows are samples, columns are SubjectID, Day, and Dose
    prefix='vandeputte_2017', # prefix of stan file name
    topN=20, # select the topN taxa to run bayesian regression
    stan_path='./'
) 

# Extract stan output and show significant fiber responses

In [4]:
fit = az.from_cmdstan(["vandeputte_2017_output_%d.csv"%(i) for i in np.arange(1,4)])

lines = []
# basal growth rate
for c in bacterial_taxa:
    var = 'alpha__%s'%(c)
    data = []
    for i in np.arange(0,3):
        data.extend(list(fit.posterior[var][i].values))
    hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
    assert len(hpd_mu) == 1
    (x0, x1) = hpd_mu[0]
    lines.append(['basal_growth_rate', c, x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])
# inulin response
for c in bacterial_taxa:
    var = 'epsilon__%s'%(c)
    data = []
    for i in np.arange(0,3):
        data.extend(list(fit.posterior[var][i].values))
    hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
    assert len(hpd_mu) == 1
    (x0, x1) = hpd_mu[0]
    lines.append(['fiber_response', c, x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])
# pairwise interactions
for c1 in bacterial_taxa:
    for c2 in bacterial_taxa:
        var = 'beta__%s_%s'%(c1,c2)
        data = []
        for i in np.arange(0,3):
            data.extend(list(fit.posterior[var][i].values))
        hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
        assert len(hpd_mu) == 1
        (x0, x1) = hpd_mu[0]
        lines.append(['pairwise_interaction', (c1,c2), x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])
    
df_stan_output_summary = pd.DataFrame(lines, columns = ['Type','Taxa','Left','Right','Middle','SNR','Significant'])
df_stan_output_summary.to_excel('bayesian_regression_summary_vandeputte_2017.xlsx')

In [5]:
# select only significant ones
df_stan_output_summary[df_stan_output_summary.Significant==True]

,Type,Taxa,Left,Right,Middle,SNR,Significant
22,fiber_response,Lachnospiraceae,0.01,0.06,0.035,2.753969,True
29,fiber_response,Anaerostipes,0.01,0.06,0.035,2.738454,True
32,fiber_response,Bifidobacterium,0.02,0.07,0.045,3.269076,True
40,pairwise_interaction,"(Bacteroides, Bacteroides)",0.49,1.92,1.205,3.298998,True
41,pairwise_interaction,"(Bacteroides, Faecalibacterium)",-7.67,-4.90,-6.285,-8.951352,True
43,pairwise_interaction,"(Bacteroides, Ruminococcus_dash_sp_dot__dash_N...",0.60,4.22,2.410,2.642690,True
46,pairwise_interaction,"(Bacteroides, Blautia)",-3.72,-0.09,-1.905,-2.074094,True
51,pairwise_interaction,"(Bacteroides, Christensenellaceae_dash_R_dash_...",0.21,3.92,2.065,2.189751,True
61,pairwise_interaction,"(Faecalibacterium, Faecalibacterium)",1.53,4.28,2.905,4.193006,True
292,pairwise_interaction,"(Bifidobacterium, Bifidobacterium)",0.36,3.93,2.145,2.360378,True
